# Supercut Template
A notebook to copy to make supercuts from. Subject to change based on experience.

In [7]:
from esper.supercuts import *
from query.models import Shot
from rekall.temporal_predicates import overlaps
from rekall.video_interval_collection import VideoIntervalCollection

In [2]:
intervals = [
    (1, 11820, 11897),
    (9, 54540, 54636),
    (24, 97476, 97596), 
    (24, 142176, 142224),
    (28, 205020, 205073),
    (32, 133080, 133155),
    (33, 19128, 19200),
    (33, 19296, 19440),
    (43, 138948, 139026),
    (47, 3204,3324),
    (47, 55824, 55860),
    (55, 112776, 113064),
    (55, 155604, 155669),
    (59, 43572, 43680),
    (59, 93384, 93408),
    (59, 126672, 126732),
    (59, 192984, 193056),
    (62, 18492, 18516),
    (62, 18851,18936),
    (62, 19392, 19447),
    (65, 121331,121652),
    (66, 171408, 171672),
    (70, 140604, 140640),
    (72, 115428, 115716),
    (72, 135436, 135600),
    (72, 138780,138793)
]

In [9]:
intervals2 = [
    (1, [(11820, 11897)]),
    (9, [(54540, 54636)]),
    (24, [(97476, 97596), (142176, 142224)]),
    (28, [(205020, 205073)]),
    (32, [(133080, 133155)]),
    (33, [(19128, 19200), (19296, 19440)]),
    (43, [(138948, 139026)]),
    (47, [(3204,3324), (55824, 55860)]),
    (55, [(112776, 113064), (155604, 155669)]),
    (59, [(43572, 43680), (93384, 93408), (126672, 126732), (192984, 193056)]),
    (62, [(18492, 18516), (18851,18936), (19392, 19447)]),
    (65, [(121331,121652)]),
    (66, [(171408, 171672)]),
    (70, [(140604, 140640)]),
    (72, [(115428, 115716), (135436, 135600), (138780,138793)])
]

In [10]:
intervals_rekall = VideoIntervalCollection({
    video_id: [
        (start, end, 0)
        for start, end in intervals
    ]
    for video_id, intervals in intervals2
})

In [13]:
shots = VideoIntervalCollection.from_django_qs(
    Shot.objects.filter(
        video_id__in=intervals_rekall.get_allintervals().keys(),
        labeler__name='shot-hsvhist-face')
)

In [26]:
kisses_gt_shots = shots.merge(intervals_rekall, predicate=overlaps()).coalesce()

In [5]:
len(intervals)

26

In [16]:
def generate_supercut_intervals(
    interval_collection,
    dilation=0,
    snap_to_cinematic_shots=False,
    limit=None,
    stride=1,
    max_length=None
):
    '''
    Generates supercut intervals for the supercut pipeline.
    Dilates interval_collection by dilation.
    If dilation > 0, snaps the dilations to cinematic shot boundaries.
    '''
    if dilation > 0:
        if snap_to_cinematic_shots:
            shots = VideoIntervalCollection.from_django_qs(
                Shot.objects.filter(
                    video_id__in=list(interval_collection.get_allintervals().keys()),
                    labeler__name="shot-hsvhist-face"
                )
            ).filter_against(interval_collection, predicate=overlaps())
        interval_collection = interval_collection.dilate(dilation).coalesce()
        if snap_to_cinematic_shots:
            interval_collection = interval_collection.overlaps(shots)
    
    supercut_intervals = []
    for video_id in sorted(list(interval_collection.get_allintervals().keys())):
        intervallist = interval_collection.get_intervallist(video_id)
        for intrvl in intervallist.get_intervals():
            if max_length is None:
                supercut_intervals.append((video_id, intrvl.get_start(), intrvl.get_end()))
            else:
                supercut_intervals.append((video_id, intrvl.get_start(), min(intrvl.get_end(), intrvl.get_start() + max_length)))
            
    if limit is not None:
        return supercut_intervals[:limit * stride:stride]
    
    return supercut_intervals[::stride]

In [27]:
supercut_intervals = generate_supercut_intervals(kisses_gt_shots)

In [28]:
len(supercut_intervals)

24

In [30]:
# output path
OUTPUT_PATH = '/app/result/supercut/kissing_ground_truth_shots.mp4'

In [31]:
stitch_video_temporal(supercut_intervals, out_path=OUTPUT_PATH, width=640, height=480)